<a href="https://colab.research.google.com/github/Eitams/NLP_Exercise/blob/main/NLP_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Upgrade profiling library and "Restart runtime"
!pip install --upgrade pandas-profiling

In [ ]:
!pip install wikipedia-api

In [8]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/NLP')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!pwd

/content/drive/MyDrive/NLP


In [5]:
import wikipediaapi
import pandas as pd
pd.set_option('display.max_columns', None)
#pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [11]:
## Loading data
df_bacteria = pd.read_csv("bacteria_df.csv", index_col=0)
df_phage = pd.read_csv("bacteriophage_df.csv", index_col=0)